In [ ]:
import matplotlib.pyplot as plt
import json
import pyLDAvis.gensim
import warnings
import seaborn as sns
from tqdm.notebook import tqdm
import os
import sys
from dotenv import load_dotenv

load_dotenv()
pyLDAvis.enable_notebook()
warnings.filterwarnings("ignore")
REPO_PATH =  os.getenv('REPO_PATH')

sys.path.insert(0, rf'{REPO_PATH}src')
from utils.text_utils import clean_token_series, IGNORE_WORDS
from utils.topic_utils import classify_article, LDAModelSetup
from utils.main_utils import combload_topic_dfs

### Import data

In [ ]:
TOPICS: list[str] = ['CRU', 'CWP', 'CEN']

text_df = combload_topic_dfs(
    TOPICS,
    lambda topic: rf'{REPO_PATH}data\news_data\EIKON_{topic}_NEWS_COMPLETE.json'
)

text_df['cleaned_tokenized'] = clean_token_series(text_df['fullStory'])

display(text_df.head(2))

### Latent Dirichlet Allocation (LDA) model setup for subtopic

In [ ]:
lda_params: dict[str, int] = {
    'num_topics': 3,
    'chunksize': 500,
    'passes': 20,
    'iterations': 100,
    'eval_every': 1
}

models = {}
for topic in TOPICS:
    model = LDAModelSetup(
        text_df.loc[text_df['topic'] == topic, 'cleaned_tokenized'],
        name=topic,
        stopwords=IGNORE_WORDS, 
        lda_params=lda_params
    )

    models[topic] = model

for topic, model in models.items():
    print(f'Creating model for {topic}...')
    model.generate_model()
    model.generate_pyLDAvis()


### LDAvis visualization of gensim LDA model

In [ ]:
TOPIC = 'CEN'

display(models[TOPIC].visfig)

In [ ]:

fig, axs = plt.subplots(1, 3, figsize=(18, 6), dpi=200)

for i, topic in enumerate(TOPICS):
    models[topic].plot_pyLDAvis(axs[i])

fig.tight_layout(pad=1)

fig.savefig(rf'images\pyLDAvis_topic_PC.png', dpi=200)

### LDA setup for cross-topic analysis

In [ ]:
lda_params: dict[str, int] = {
    'num_topics': 5,
    'chunksize': 1000,
    'passes': 30,
    'iterations': 500,
    'eval_every': 1
}

full_model = LDAModelSetup(
    text_df['cleaned_tokenized'],
    name='All topics',
    stopwords=IGNORE_WORDS, 
    lda_params=lda_params
)

full_model.generate_model()
full_model.generate_pyLDAvis()

display(full_model.visfig)

In [ ]:
tqdm.pandas()
df = text_df.copy()
df['crosstopic'] = df.progress_apply(
    lambda x: classify_article(
        x, 
        full_model.dictionary, 
        full_model.model
    ), axis=1
)

In [ ]:
topic_names: dict[int, str] = {
    0: 'Geopolitical Conflicts',
    1: 'Interest Rates and Economic Policy',
    2: 'Oil and Gas Production',
    3: 'Banking and Finance',
    4: 'Securities and Commodity Markets'
}

fig, ax = plt.subplots(1, 5, figsize=(16, 3), dpi=200)

colors = sns.color_palette('twilight', n_colors=3)
order = []

size_df = df['crosstopic'].value_counts().sort_index()
size_df = size_df.reindex(size_df.sort_values(ascending=False).index)

for i, topic in enumerate(size_df.index.values):
    # print topic size in %
    values = df[df['crosstopic'] == topic]['topic'].value_counts().reindex(TOPICS)
    values.plot.pie(ax=ax[i], colors=colors)

    ax[i].set_ylabel('')
    ax[i].set_title(r'$\bf{Topic}$' + f' {i + 1}\n{topic_names[i]}', fontsize=11)

    # print top 10 words for each topic
    top_words = full_model.model.show_topic(topic, topn=20)
    words = [word for word, _ in top_words]
    order.append(words)
    print(f'Topic {i + 1}: {words}')

fig.tight_layout(h_pad=1)

fig.savefig(rf'images\pyLDAvis_all_crosstopics.png')

### PCA visualization of LDA model



In [ ]:
fig, ax = plt.subplots(figsize=(7,7), dpi=200)

full_model.plot_pyLDAvis(ax)

fig.savefig(rf'images\pyLDAvis_crosstopic_PCA.png')

### Assign topics to each document

In [ ]:

for topic in TOPICS:
    df_temp = text_df[text_df['topic'] == topic]
    tqdm.pandas(desc=f"Adding subtopics to {topic}")
    df_temp['topic'] = df_temp.progress_apply(
        lambda x: classify_article(
            x,
            models[topic].dictionary, 
            models[topic].model
        ), axis=1
    )
    topic_dict = dict(zip(df_temp['storyId'], df_temp['topic']))

    with open(rf'{REPO_PATH}data\topic_data\{topic}_TOPICS.json', 'w') as f:
        json.dump(topic_dict, f, indent=2)


cross_topic_dict = dict(zip(df['storyId'], df['crosstopic']))

with open(rf'{REPO_PATH}data\topic_data\CROSS_TOPICS.json', 'w') as f:
        json.dump(cross_topic_dict, f, indent=2)
